In [1]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
file_path = "./batch_qyk7ROhHeTyXmjdN2KfmYnTP_output.jsonl"
gt_data = pd.read_json(f"./generated_images_varied_thickness/config.json")

In [3]:
# Dictionary to map custom_id to message content
custom_id_to_content = {}

# Open the file and read line by line
with open(file_path, "r") as file:
    for line in file:
        json_obj = json.loads(line)
        # Extract custom_id and message content
        custom_id = json_obj.get("custom_id")
        message_content = json_obj["response"]["body"]["choices"][0]["message"][
            "content"
        ]
        # Map custom_id to message content
        if custom_id:
            custom_id_to_content[custom_id.replace("uid__", "")] = message_content

# Print the dictionary to verify
print(len(custom_id_to_content))

150


In [4]:
custom_id_to_content

{'image_0_thickness_2.png': 'There are two line intersections in the image.',
 'image_0_thickness_3.png': 'There is one line intersection in the image where the blue and red lines cross each other.',
 'image_0_thickness_4.png': 'There is one intersection point where the red and blue lines cross each other in the image.',
 'image_1_thickness_2.png': 'There is one line intersection in the image, where the blue and red lines intersect.',
 'image_1_thickness_3.png': 'There is one intersection point in the image where the red and blue lines cross each other.',
 'image_1_thickness_4.png': 'There is one intersection between the blue lines and the red line in the image.',
 'image_2_thickness_2.png': 'There are two line intersections in the image.',
 'image_2_thickness_3.png': 'There is one intersection point between the blue and the red lines in the image.',
 'image_2_thickness_4.png': 'There are two line intersections in the image. The red line intersects the blue line in two places.',
 'imag

In [5]:
gt_data = (
    gt_data.T
)  # use indexx as a new column in the dataframe and called it image_path
gt_data = gt_data.reset_index()
gt_data = gt_data.rename(columns={"index": "image_path"})
gt_data

,image_path,num_points,num_intersections,line_colors,line_thickness
0,image_0_thickness_2.png,3,1,"[red, blue]",2
1,image_0_thickness_3.png,3,1,"[red, blue]",3
2,image_0_thickness_4.png,3,1,"[red, blue]",4
3,image_1_thickness_2.png,3,1,"[red, blue]",2
4,image_1_thickness_3.png,3,1,"[red, blue]",3
...,...,...,...,...,...
145,image_48_thickness_3.png,3,2,"[red, blue]",3
146,image_48_thickness_4.png,3,2,"[red, blue]",4
147,image_49_thickness_2.png,3,1,"[red, blue]",2
148,image_49_thickness_3.png,3,1,"[red, blue]",3


In [6]:
# ad a new column to gt_data with content of  custom_id_to_content (match key with image_path)
gt_data["message"] = gt_data["image_path"].map(custom_id_to_content)

In [7]:
import re


# Function to extract numbers from text, including zero represented as text
def extract_number(text):
    # Check for phrases that imply zero intersections
    if (
        "no intersections" in text
        or "zero line intersections" in text
        or "no line" in text
    ):
        return 0
    # First try to find digits
    match = re.search(r"\b\d+\b", text)
    if match:
        return int(match.group())
    # If no digits, look for text numbers (one, two, three, etc.)
    text_to_num = {"one": 1, "two": 2, "three": 3, "four": 4, "five": 5}
    for word, num in text_to_num.items():
        if word in text:
            return num
    return None


# Apply the function to the 'message' column
gt_data["extracted_number"] = gt_data["message"].apply(extract_number)
gt_data[["image_path", "message", "extracted_number"]]

,image_path,message,extracted_number
0,image_0_thickness_2.png,There are two line intersections in the image.,2
1,image_0_thickness_3.png,There is one line intersection in the image wh...,1
2,image_0_thickness_4.png,There is one intersection point where the red ...,1
3,image_1_thickness_2.png,"There is one line intersection in the image, w...",1
4,image_1_thickness_3.png,There is one intersection point in the image w...,1
...,...,...,...
145,image_48_thickness_3.png,There are three line intersections in the image.,3
146,image_48_thickness_4.png,There are three intersections in the image whe...,3
147,image_49_thickness_2.png,There are two line intersections in the image.,2
148,image_49_thickness_3.png,There are 2 line intersections in the image.,2


In [8]:
gt_data

,image_path,num_points,num_intersections,line_colors,line_thickness,message,extracted_number
0,image_0_thickness_2.png,3,1,"[red, blue]",2,There are two line intersections in the image.,2
1,image_0_thickness_3.png,3,1,"[red, blue]",3,There is one line intersection in the image wh...,1
2,image_0_thickness_4.png,3,1,"[red, blue]",4,There is one intersection point where the red ...,1
3,image_1_thickness_2.png,3,1,"[red, blue]",2,"There is one line intersection in the image, w...",1
4,image_1_thickness_3.png,3,1,"[red, blue]",3,There is one intersection point in the image w...,1
...,...,...,...,...,...,...,...
145,image_48_thickness_3.png,3,2,"[red, blue]",3,There are three line intersections in the image.,3
146,image_48_thickness_4.png,3,2,"[red, blue]",4,There are three intersections in the image whe...,3
147,image_49_thickness_2.png,3,1,"[red, blue]",2,There are two line intersections in the image.,2
148,image_49_thickness_3.png,3,1,"[red, blue]",3,There are 2 line intersections in the image.,2


In [9]:
# count none or na in gt_data extracted_number
gt_data["extracted_number"].isna().sum()
# show their value in the message (same row)
gt_data[gt_data["extracted_number"].isna()]["message"]

Series([], Name: message, dtype: object)

In [10]:
# Cast num_intersections to integer
gt_data["num_intersections"] = gt_data["num_intersections"].astype(int)

# Cast extracted_number to integer (it should already be integer from the extraction function, but just to ensure consistency)
gt_data["extracted_number"] = gt_data["extracted_number"].astype(int)

# Compare num_intersections and extracted_number to check if they are the same
gt_data["is_correct"] = gt_data["num_intersections"] == gt_data["extracted_number"]

# Display the DataFrame with the new column
gt_data[["image_path", "num_intersections", "extracted_number", "is_correct"]]

,image_path,num_intersections,extracted_number,is_correct
0,image_0_thickness_2.png,1,2,False
1,image_0_thickness_3.png,1,1,True
2,image_0_thickness_4.png,1,1,True
3,image_1_thickness_2.png,1,1,True
4,image_1_thickness_3.png,1,1,True
...,...,...,...,...
145,image_48_thickness_3.png,2,3,False
146,image_48_thickness_4.png,2,3,False
147,image_49_thickness_2.png,1,2,False
148,image_49_thickness_3.png,1,2,False


In [11]:
gt_data["is_correct"].value_counts()

is_correct
False    85
True     65
Name: count, dtype: int64

In [12]:
100 * np.mean(gt_data["is_correct"])

43.333333333333336

In [13]:
gt_data

,image_path,num_points,num_intersections,line_colors,line_thickness,message,extracted_number,is_correct
0,image_0_thickness_2.png,3,1,"[red, blue]",2,There are two line intersections in the image.,2,False
1,image_0_thickness_3.png,3,1,"[red, blue]",3,There is one line intersection in the image wh...,1,True
2,image_0_thickness_4.png,3,1,"[red, blue]",4,There is one intersection point where the red ...,1,True
3,image_1_thickness_2.png,3,1,"[red, blue]",2,"There is one line intersection in the image, w...",1,True
4,image_1_thickness_3.png,3,1,"[red, blue]",3,There is one intersection point in the image w...,1,True
...,...,...,...,...,...,...,...,...
145,image_48_thickness_3.png,3,2,"[red, blue]",3,There are three line intersections in the image.,3,False
146,image_48_thickness_4.png,3,2,"[red, blue]",4,There are three intersections in the image whe...,3,False
147,image_49_thickness_2.png,3,1,"[red, blue]",2,There are two line intersections in the image.,2,False
148,image_49_thickness_3.png,3,1,"[red, blue]",3,There are 2 line intersections in the image.,2,False


In [19]:
# Assuming 'gt_data' DataFrame has columns 'line_thickness', 'num_intersections', and 'extracted_number'

# Convert line_thickness to an appropriate numeric type if necessary
gt_data["line_thickness"] = gt_data["line_thickness"].astype(int)

# Calculate accuracy for each thickness
accuracy_by_thickness = gt_data.groupby("line_thickness").apply(
    lambda df: (df["num_intersections"] == df["extracted_number"]).mean()
)

# Print the accuracy by line thickness
# print(accuracy_by_thickness
#
# #)
# multiply by 100 and round 2 decimal
accuracy_by_thickness = 100 * accuracy_by_thickness.round(2)

/var/folders/y1/b5wjnkp96_b7cvzncx0zsry40000gn/T/ipykernel_30518/1027132002.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  accuracy_by_thickness = gt_data.groupby("line_thickness").apply(


In [20]:
print(accuracy_by_thickness.to_latex())

\begin{tabular}{lr}
\toprule
 & 0 \\
line_thickness &  \\
\midrule
2 & 40.000000 \\
3 & 42.000000 \\
4 & 48.000000 \\
\bottomrule
\end{tabular}

